In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter

In [2]:
df3 = pd.read_csv('../ess/ess3/ess3.csv')
df4 = pd.read_csv('../ess/ess4/ess4.csv')
df5 = pd.read_csv('../ess/ess5/ess5.csv')
df6 = pd.read_csv('../ess/ess6/ess6.csv')
df7 = pd.read_csv('../ess/ess7/ess7.csv')
df8 = pd.read_csv('../ess/ess8/ess8.csv')
df9 = pd.read_csv('../ess/ess9/ess9.csv')
df10 = pd.read_csv('../ess/ess10/ess10.csv')
df10S = pd.read_csv('../ess/ess10S/ess10S.csv')
# print(df.head())

dataframes = {'df3': df3, 'df4': df4, 'df5': df5, 'df6': df6, 'df7': df7, 'df8': df8, 'df9': df9, 'df10': df10, 'df10S': df10S}

In [3]:
# Define the columns to clean
# columns_to_clean = ['lrscale', 'stfdem', 'stfeco', 'stfedu', 'stfgov', 'stfhlth', 'stflife', 'trstep', 'trstplt', 'trstprl', 'trstprt', 'atchctr', 'atcherp', 'happy']
columns_to_clean = ['lrscale', 'stfdem', 'stfeco', 'stfedu', 'stfgov', 'stfhlth', 'stflife', 'trstep', 'trstplt', 'trstprl', 'trstprt', 'happy']

# Define the values to remove
values_to_remove = [77, 88, 99, '*', np.nan]

cleaned_dfs = []
for name, df in dataframes.items():
    for column in columns_to_clean:
        df = df[~df[column].isin(values_to_remove)]
    cleaned_dfs.append(df)
    # print(f"{name} - {df['name']}")
df = pd.concat(cleaned_dfs, ignore_index=True)

In [4]:
# Define bins and labels
bins = [-1, 0, 2, 4, 5, 7, 9, 10]
labels = ['far-left', 'left', 'center-left', 'center', 'center-right', 'right', 'far-right']

# Create lrscale_category column based on bins and labels
df['leaning'] = pd.cut(df['lrscale'], bins=bins, labels=labels)

# year
version_year_dict = {3: "2006", 4: "2008", 5: "2010", 6: "2012", 7: "2014", 8: "2016", 9: "2018", 10: "2020"}
df['year'] = None
for i, row in df.iterrows():
    version_str = row['name'].split('ESS')[1].split('e')[0]
    version = 10 if version_str == "10SC" else int(version_str)
    year = version_year_dict.get(version)
    df.at[i,'year'] = year

# satisfaction
satisfaction_columns = ['stfdem', 'stfeco', 'stfedu', 'stfgov', 'stfhlth', 'stflife']
df['satisfaction'] = np.round(df[satisfaction_columns].mean(axis=1),0)

# trust country
trust_columns = ['trstplt', 'trstprl', 'trstprt']
df['trust_country'] = np.round(df[trust_columns].mean(axis=1),0)

# trust eu
df['trust_eu'] = df['trstep']

# # attachment country
# df['attachment_country'] = df['atchctr']

# # attachment eu
# df['attachment_eu'] = df['atcherp']

# new_df = df[['year','cntry','leaning','happy','satisfaction','trust_country','trust_eu','attachment_country', 'attachment_eu']]
new_df = df[['year','cntry','leaning','happy','satisfaction','trust_country','trust_eu']]


In [5]:
be_years = new_df[new_df['cntry'] == 'BE']['year'].unique()
print(be_years)

['2006' '2008' '2010' '2012' '2014' '2016' '2018' '2020']


In [6]:
new_df.loc[new_df['cntry'] == 'GR', 'cntry'] = 'EL'

In [7]:
with open("../json/combined_data.json") as f:
    data = json.load(f)

In [8]:
# Initialize 'ess_data' in each country
for country in data:
    data[country]['ess_data'] = {}
print(data['BE'])

{'election_data': {'2019': {'NVA': 16.0, 'VB': 12.0, 'PS - Belgium': 9.5, 'CDV': 8.9, 'PVDA - Belgium': 8.6, 'OVLD': 8.5, 'MR': 7.6, 'SPA': 6.7, 'Ecolo': 6.1, 'Groen': 6.0, 'CDH': 3.7, 'Defi': 2.2, 'PP - Belgium': 1.1, 'Other parties': 3.1}, '2014': {'NVA': 20.3, 'PS - Belgium': 11.7, 'CDV': 11.6, 'OVLD': 9.8, 'MR': 9.6, 'SPA': 8.8, 'Groen': 5.3, 'CDH': 5.0, 'VB': 3.7, 'PVDA - Belgium': 3.7, 'Ecolo': 3.3, 'DeFI': 1.8, 'PP - Belgium': 1.5, 'LD': 0.4, 'Other parties': 3.5}, '2010': {'NVA': 17.4, 'PS - Belgium': 13.7, 'CDV': 10.9, 'MR': 9.3, 'SPA': 9.2, 'OVLD': 8.6, 'VB': 7.8, 'CDH': 5.5, 'Ecolo': 4.8, 'Groen': 4.4, 'LD': 3.7, 'PVDA - Belgium': 1.6, 'PP - Belgium': 1.3, 'FN - Belgium': 0.5, 'Other parties': 1.3}, '2007': {'CDV/NVA': 18.5, 'MR': 12.5, 'VB': 12.0, 'OVLD': 11.8, 'PS - Belgium': 10.9, 'SPA': 10.3, 'CDH': 6.1, 'Ecolo': 5.1, 'LD': 4.0, 'Groen': 4.0, 'FN - Belgium': 2.0, 'PVDA - Belgium': 0.8, 'Other parties': 2.0}, '2003': {'OVLD': 15.3, 'SPA': 14.9, 'CDV': 13.2, 'PS - Belgium'

In [9]:
# Iterate over the DataFrame
for index, row in new_df.iterrows():
    # Get country and year
    country = row['cntry']
    year = row['year']

    # Initialize year in 'ess_data' if not already present
    if year not in data[country]['ess_data']:
        data[country]['ess_data'][year] = {
            'leaning': Counter(),
            'happy': Counter(),
            'satisfaction': Counter(),
            'trust_country': Counter(),
            'trust_eu': Counter()
        }

    # Append data to 'ess_data'
    data[country]['ess_data'][year]['leaning'].update([row['leaning']])
    data[country]['ess_data'][year]['happy'].update([row['happy']])
    data[country]['ess_data'][year]['satisfaction'].update([row['satisfaction']])
    data[country]['ess_data'][year]['trust_country'].update([row['trust_country']])
    data[country]['ess_data'][year]['trust_eu'].update([row['trust_eu']])

In [10]:
from collections import OrderedDict
# Sort 'ess_data' by year
for country in data:
    data[country]['ess_data'] = OrderedDict(sorted(data[country]['ess_data'].items(), reverse=True))

In [11]:
for country in data:
    for year in data[country]['ess_data']:
        print(f"{country} {year}: {len(data[country]['ess_data'][year]['happy'])}")
    print()

AT 2020: 11
AT 2018: 11
AT 2016: 10
AT 2014: 11
AT 2006: 11

BE 2020: 11
BE 2018: 11
BE 2016: 11
BE 2014: 10
BE 2012: 11
BE 2010: 10
BE 2008: 11
BE 2006: 11

DE 2020: 11
DE 2018: 11
DE 2016: 11
DE 2014: 11
DE 2012: 11
DE 2010: 11
DE 2008: 11
DE 2006: 11

DK 2018: 10
DK 2014: 11
DK 2012: 11
DK 2010: 11
DK 2008: 9
DK 2006: 11

EL 2020: 11
EL 2010: 11
EL 2008: 11

ES 2020: 11
ES 2018: 11
ES 2016: 11
ES 2014: 11
ES 2012: 11
ES 2010: 11
ES 2008: 10
ES 2006: 11

FI 2020: 11
FI 2018: 10
FI 2016: 11
FI 2014: 11
FI 2012: 10
FI 2010: 10
FI 2008: 11
FI 2006: 11

FR 2020: 11
FR 2018: 11
FR 2016: 11
FR 2014: 11
FR 2012: 11
FR 2010: 11
FR 2008: 11
FR 2006: 11

IE 2020: 10
IE 2018: 11
IE 2016: 11
IE 2014: 11
IE 2012: 11
IE 2010: 11
IE 2008: 11
IE 2006: 11

IT 2020: 11
IT 2018: 11
IT 2016: 11
IT 2012: 11


NL 2020: 11
NL 2018: 11
NL 2016: 11
NL 2014: 11
NL 2012: 11
NL 2010: 11
NL 2008: 10
NL 2006: 11

PT 2020: 11
PT 2018: 11
PT 2016: 11
PT 2014: 11
PT 2012: 11
PT 2010: 11
PT 2008: 11
PT 2006: 11

SE 2

In [12]:
with open('ess_data3.json', 'w') as f:
    json.dump(data, f, indent=3)